In [330]:
%matplotlib inline

import colorsys
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from bokeh.plotting import figure, show
from bokeh.resources import CDN
from bokeh.io import output_notebook
output_notebook( resources=CDN )
import math

pd.set_option('max_colwidth', 200)

Loading BokehJS ...

In [331]:
# Tube
lines       = pd.read_csv('london.lines.csv', index_col=0)
stations    = pd.read_csv('london.stations.csv', index_col=0)
stationsRein= pd.read_csv('london.stations.reindexed.csv', index_col=0)
connections = pd.read_csv('london.connections.csv')

# Bikes
bikesRawData       = pd.read_csv('bikes/london-bikes-coords.csv', index_col=0)

In [332]:
tubeGraph = nx.Graph()

for connection_id, connection in connections.iterrows():
    station1_name = stations.ix[connection['station1']]['name']
    station2_name = stations.ix[connection['station2']]['name']
    tubeGraph.add_edge(station1_name, station2_name, time = connection['time'])
    
#add the connection between Bank and Monument manually
tubeGraph.add_edge('Bank', 'Monument', time = 1)
# nx.draw(tubeGraph)

/usr/local/Cellar/ipython/6.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/usr/local/Cellar/ipython/6.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [333]:
nx.shortest_path(tubeGraph, 'Oxford Circus', 'Canary Wharf', weight='time')

['Oxford Circus',
 'Tottenham Court Road',
 'Holborn',
 'Chancery Lane',
 "St. Paul's",
 'Bank',
 'Shadwell',
 'Wapping',
 'Rotherhithe',
 'Canada Water',
 'Canary Wharf']

In [334]:
# Print Bike Data
bikesRawData

,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndLatitude,EndLongitude,StartLatitude,StartLongitude
Rental Id,,,,,,,,,,,,
73667188,720,7315,6/3/18 7:18,282,"Royal London Hospital, Whitechapel",6/3/18 7:06,30,"Windsor Terrace, Hoxton",51.518222,-0.058518,51.529239,-0.093476
73693268,540,3500,6/3/18 23:14,95,"Aldersgate Street, Barbican",6/3/18 23:05,751,"Newton Street, Covent Garden",51.520091,-0.097608,51.516573,-0.121343
73627763,1560,6139,3/3/18 20:32,60,"Lisson Grove, St. John's Wood",3/3/18 20:06,203,"West Smithfield Rotunda, Farringdon",51.524779,-0.168315,51.518248,-0.100737
73638919,480,12516,4/3/18 20:04,107,"Finsbury Leisure Centre, St. Luke's",4/3/18 19:56,203,"West Smithfield Rotunda, Farringdon",51.526022,-0.095231,51.518248,-0.100737
73623157,1320,7203,3/3/18 12:56,496,"Devons Road, Bow",3/3/18 12:34,446,"York Hall, Bethnal Green",51.522163,-0.018298,51.530140,-0.055168
73607211,540,3959,1/3/18 14:29,521,"Driffield Road, Old Ford",1/3/18 14:20,446,"York Hall, Bethnal Green",51.533171,-0.036762,51.530140,-0.055168
73592503,900,14948,28/02/2018 07:25,508,"Fournier Street, Whitechapel",28/02/2018 07:10,70,"Calshot Street , King's Cross",51.519277,-0.073497,51.532791,-0.117255
73675110,300,14663,6/3/18 9:34,136,"Queen Victoria Street, St. Paul's",6/3/18 9:29,95,"Aldersgate Street, Barbican",51.511713,-0.099315,51.520091,-0.097608
73678145,120,7194,6/3/18 12:29,339,"Risinghill Street, Angel",6/3/18 12:27,234,"Liverpool Road (N1 Centre), Angel",51.533199,-0.112322,51.534534,-0.106988


In [335]:
normed = stations[['longitude', 'latitude']]
normed = normed - normed.min()
normed = normed / normed.max()
locations = dict(zip(stations['name'], normed[['longitude', 'latitude']].values))

p = figure(
    x_range = (.4,.7),
    y_range = (.2,.5),
    height= 700,
    width= 900,
)
for edge in tubeGraph.edges():
    p.line( 
        x= [locations[pt][0] for pt in edge],
        y= [locations[pt][1] for pt in edge],
    )

for node in tubeGraph.nodes():
    x = [locations[node][0]]
    y = [locations[node][1]]
    p.circle(
        x, y, 
        line_alpha=0)
    p.text(
        x, y, 
        text = {'value':node}, 
        text_font_size = str(10) +"pt",
        text_align='center',
        text_font_style='bold')
    
show(p)

In [336]:
nx.algorithms.connectivity.cuts.minimum_node_cut(tubeGraph)

{'Warwick Avenue'}

In [337]:
nx.algorithms.connectivity.stoerwagner.stoer_wagner(tubeGraph)

(1,
 (['Upminster'],
  ['Chiswick Park',
   'Stamford Brook',
   'Bond Street',
   'West India Quay',
   'Northwick Park',
   'Arnos Grove',
   'Highbury & Islington',
   'Island Gardens',
   'South Ruislip',
   'Canons Park',
   'Wimbledon Park',
   'West Hampstead',
   'Camden Town',
   'Stockwell',
   'Hounslow West',
   'Woodside Park',
   'Moorgate',
   'North Acton',
   'Aldgate',
   'Beckton',
   'Elverson Road',
   'Putney Bridge',
   'New Cross Gate',
   'Cockfosters',
   'Bromley-By-Bow',
   'South Woodford',
   'Bounds Green',
   'Richmond',
   'Upminster Bridge',
   'Colindale',
   'Highgate',
   'West Ham',
   'Morden',
   'Southfields',
   'Tottenham Court Road',
   'Brixton',
   'Warren Street',
   'Beckton Park',
   "King's Cross St. Pancras",
   'Upton Park',
   'South Kensington',
   'Alperton',
   'Wanstead',
   'Becontree',
   'Plaistow',
   'Canary Wharf',
   'Upney',
   'Hounslow East',
   "Shepherd's Bush (C)",
   'Hampstead',
   'Fairlop',
   'Chalk Farm',
   'R

In [533]:
# BIKES
uniqueBikeLocations = {}
uniqueBikeCount = 0

for id, info in bikesRawData.iterrows():
    if info["StartStation Name"] not in uniqueBikeLocations:
        name = info["StartStation Name"]
        lat = info["StartLatitude"]
        long = info["StartLongitude"]
        uniqueBikeLocations[name] = (lat, long)
        uniqueBikeCount += 1
        
    if info["EndStation Name"] not in uniqueBikeLocations:
        name = info["EndStation Name"]
        lat = info["EndLatitude"]
        long = info["EndLongitude"]
        uniqueBikeLocations[name] = (lat, long)
        uniqueBikeCount += 1
        
print("UniqueBikeCount: ",uniqueBikeCount)


UniqueBikeCount:  733
['Windsor Terrace, Hoxton', 'Newton Street, Covent Garden', 'West Smithfield Rotunda, Farringdon', 'West Smithfield Rotunda, Farringdon', 'York Hall, Bethnal Green', 'York Hall, Bethnal Green', "Calshot Street , King's Cross", 'Aldersgate Street, Barbican', 'Liverpool Road (N1 Centre), Angel', 'Liverpool Road (N1 Centre), Angel', 'Frampton Street, Paddington', 'Frampton Street, Paddington', 'Frampton Street, Paddington', 'Ebury Bridge, Pimlico', 'Ebury Bridge, Pimlico', 'Tallis Street, Temple', 'Tallis Street, Temple', "Lisson Grove, St. John's Wood", "Lisson Grove, St. John's Wood", 'Alderney Street, Pimlico', "St. George's Square, Pimlico", 'Goswell Road (City Uni), Finsbury', 'Aldersgate Street, Barbican', 'Walmer Road, Avondale', 'Freston Road, Avondale', 'Chepstow Villas, Notting Hill', 'Derry Street, Kensington', 'Princedale Road , Holland Park', 'Lansdowne Road, Ladbroke Grove', 'Torrens Street, Angel', 'Leonard Circus , Shoreditch', 'Finlay Street, Fulham'

In [339]:
# list of unique bike locations ("stations") and their coordinates
pd.DataFrame.from_dict(uniqueBikeLocations, orient="index", columns=['latitude', 'longitude'])

,latitude,longitude
"Windsor Terrace, Hoxton",51.529239,-0.093476
"Royal London Hospital, Whitechapel",51.518222,-0.058518
"Newton Street, Covent Garden",51.516573,-0.121343
"Aldersgate Street, Barbican",51.520091,-0.097608
"West Smithfield Rotunda, Farringdon",51.518248,-0.100737
"Lisson Grove, St. John's Wood",51.524779,-0.168315
"Finsbury Leisure Centre, St. Luke's",51.526022,-0.095231
"York Hall, Bethnal Green",51.530140,-0.055168
"Devons Road, Bow",51.522163,-0.018298
"Driffield Road, Old Ford",51.533171,-0.036762


In [492]:
###################### START OF THE COMPUTATION #####################

In [493]:
# ------------- Get Random value and Mean -----------------

def randomCoordinate():
    # Calcuates mean coordinates based on Tube Stations
    latcoors = stationsRein['latitude']
    longcoors = stationsRein['longitude']
    latmean = latcoors.mean()
    longmean = longcoors.mean()

    # meancoors = (latmean, longmean)
    meancoors = (51.510776, -0.115638) # Center of London
    # print("Mean Coordinates: ", meancoors)

    longitudeBoundSize = 0.2 # Mess with these for bigger radius
    latitudeBoundSize = 0.2  # Mess with these for bigger radius

    randLat1 = meancoors[0] + (random.uniform(0,1)*latitudeBoundSize - (latitudeBoundSize/2))
    randLong1 = meancoors[1] + (random.uniform(0,1)*longitudeBoundSize - (longitudeBoundSize/2))
    randLat2 = meancoors[0] + (random.uniform(0,1)*latitudeBoundSize - (latitudeBoundSize/2))
    randLong2 = meancoors[1] + (random.uniform(0,1)*longitudeBoundSize - (longitudeBoundSize/2))

    print("Random origin location:        ", randLat1,"," ,randLong1)
    print("Random destination location:   ", randLat2,",", randLong2)
    
    return randLat1, randLong1, randLat2, randLong2

In [494]:
# --------------- Calculates the closest TUBE stations to the random coordinates --------------

def closestTubeStation(randLat1, randLong1, randLat2, randLong2):
    #calculate distances to all stations in array from co-ordinates
    tubeDistances1 = []
    tubeDistances2 = []
    for i in range(1, len(stationsRein['longitude'])):
        distance1 = math.sqrt(((randLat1 - stationsRein['latitude'].iloc[i-1])*(11/7))**2 + (randLong1 - stationsRein['longitude'].iloc[i-1])**2)
        distance2 = math.sqrt(((randLat2 - stationsRein['latitude'].iloc[i-1])*(11/7))**2 + (randLong2 - stationsRein['longitude'].iloc[i-1])**2)

        tubeDistances1.append(distance1)
        tubeDistances2.append(distance2)

    #pick shortest for each co-ordinate
    station1 = tubeDistances1.index(min(tubeDistances1))
    station2 = tubeDistances2.index(min(tubeDistances2))
    
    oName = stationsRein['name'][station1+1]
    oLat  = stationsRein['latitude'][station1+1]
    oLong = stationsRein['longitude'][station1+1]
    
    dName = stationsRein['name'][station2+1]
    dLat  = stationsRein['latitude'][station2+1]
    dLong = stationsRein['longitude'][station2+1]

    print("Nearest train station to Random Origin:",oName,oLat,oLong)
    print("Nearest train station to Random Destination:",dName,dLat,dLong)
    
    return oName, dName
    #return oName,oLat,oLong,dName,dLat,dLong

In [495]:
# ---------------- Calculate the closest BIKE station to the random coordinates ---------------

def closestBikeStation(randLat1, randLong1, randLat2, randLong2):
    # Calculate distances to all stations in array from co-ordinates
    shortestBikeOriginName = ""
    shortestBikeDestinationName = ""
    shortestBikeOriginValue = 9999
    shortestBikeDestinationValue = 9999

    for bikeLoc in uniqueBikeLocations:
        bName = bikeLoc
        bLat  = uniqueBikeLocations[bikeLoc][0]
        bLong = uniqueBikeLocations[bikeLoc][1]
        distanceFromRandomToOrigin      = math.sqrt(((randLat1 - bLat)*11/7)**2 + (randLong1 - bLong)**2)
        distanceFromRandomToDestination = math.sqrt(((randLat2 - bLat)*11/7)**2 + (randLong2 - bLong)**2)

        if distanceFromRandomToOrigin < shortestBikeOriginValue:
            shortestBikeOriginName = bName
            shortestBikeOriginValue = distanceFromRandomToOrigin

        if distanceFromRandomToDestination < shortestBikeDestinationValue:
            shortestBikeDestinationName = bName
            shortestBikeDestinationValue = distanceFromRandomToDestination

    print("Nearest bike station to Random Origin:", shortestBikeOriginName, uniqueBikeLocations[shortestBikeOriginName])
    print("Nearest bike station to Random Destination:", shortestBikeDestinationName, uniqueBikeLocations[shortestBikeDestinationName])
    
    return shortestBikeOriginName, uniqueBikeLocations[shortestBikeOriginName], shortestBikeDestinationName, uniqueBikeLocations[shortestBikeDestinationName]

In [496]:
# ------------- Calculate BIKE travel time -----------------------
# Calculates the mean of the closest bike ports, as well as ones that are really close.

def calcBikeTravelTimeMins(oBike, oBikeCoords, dBike, dBikeCoords):
    exactMatches = 0
    relativeMatches = 0
    totalDurationAllTrips = 0
    
    for entry_id, entry in bikesRawData.iterrows():
        eStartName =  entry["StartStation Name"]
        eStartCoords = (entry["StartLatitude"], entry["StartLongitude"])

        eEndName   =  entry["EndStation Name"]
        eEndCoords = (entry["EndLatitude"], entry["EndLongitude"])

        eDuration = entry["Duration"]

        if oBike == eStartName and dBike == eEndName :
            exactMatches += 1
            totalDurationAllTrips += eDuration
            continue

        #   Also include trips from really nearby stations..
        startLatEp = abs(eStartCoords[0] - oBikeCoords[0]) < 0.02
        startLongEp = abs(eStartCoords[1] - oBikeCoords[1]) < 0.02
        endLatEp = abs(eEndCoords[0] - dBikeCoords[0]) < 0.02
        endLatEp = abs(eEndCoords[1] - dBikeCoords[1]) < 0.02
        #  print(startLatEp, startLongEp, endLatEp, endLatEp)
        if startLatEp and startLongEp and endLatEp and endLatEp:
            relativeMatches += 1
            totalDurationAllTrips += eDuration
            
            
    print("For bikes there were:",exactMatches, "exact matches.")
    print("For bikes there were:",relativeMatches, "relative matches.")
    #print(totalDurationAllTrips / (exactMatches + relativeMatches))
    
    if exactMatches + relativeMatches == 0:
        return -1 # ERROR! No matches...
    
    bikeMins = (totalDurationAllTrips / (exactMatches + relativeMatches))/60
    print("The average time of this BIKE trip is:", bikeMins, "minutes.")
    return bikeMins

In [519]:
# --------------- Calculate TRAIN travel time --------------
# Calculates with dijkstra's shortest path algorithm (weighted by TIME)
def calcTrainTravelTimeMins(oName, dName):
    
    nx.algorithms.shortest_paths.dijkstra_path(tubeGraph, oName, dName)

    quickestTrainTime = nx.algorithms.shortest_paths.dijkstra_path_length(tubeGraph,oName,dName, weight = "time")

    print("The quickest time for this TRAIN trip is ",quickestTrainTime, "minutes.")
    
    return quickestTrainTime

In [530]:
########## RUN ME ############
output = {}
for i in range(0, 50):
    print("[!] Iteration", i)
    # Compute random coordinates in our range
    (randLat1, randLong1, randLat2, randLong2) = randomCoordinate()

    # Calculate the nearest origin and destination stations, given the random coordinates.
    (oTrain,dTrain) = closestTubeStation(randLat1, randLong1, randLat2, randLong2)
    (oBike, oBikeCoords, dBike, dBikeCoords) = closestBikeStation(randLat1, randLong1, randLat2, randLong2)

    # Calculate the minimum/average travel time (with our data) given an origin and a destination.
    bikeMins = calcTrainTravelTimeMins(oTrain,dTrain)
    trainMins = calcBikeTravelTimeMins(oBike, oBikeCoords, dBike, dBikeCoords)

    if bikeMins == -1 or trainMins == -1:
        print("error")
        continue
        
    output[i] = {"oRandomLat": randLat1,
                 "oRandomLong": randLong1,
                 "dRandomLat": randLat2,
                 "dRandomLong": randLong2,
                 "oTrain": oTrain, 
                 "dTrain": dTrain,
                 "oBike": oBike,
                 "oBikeCoords":oBikeCoords,
                 "dBike": dBike,
                 "dBikeCoords":dBikeCoords,
                 "bikeMins": bikeMins,
                 "trainMins": trainMins}
    
# print(output[0])



[!] Iteration 0
Random origin location:         51.58632573030417 , -0.18225777506296154
Random destination location:    51.42103535013567 , -0.1469035929473643
Nearest train station to Random Origin: East Finchley 51.5874 -0.165
Nearest train station to Random Destination: Tooting Broadway 51.4275 -0.168
Nearest bike station to Random Origin: Allington street, Westminster (51.5806125, -0.2282815)
Nearest bike station to Random Destination: Spencer Park, Wandsworth Common (51.45316469999999, -0.169008)
The quickest time for this TRAIN trip is  38 minutes.
For bikes there were: 0 exact matches.
For bikes there were: 48 relative matches.
The average time of this BIKE trip is: 13.5625 minutes.
[!] Iteration 1
Random origin location:         51.46100904331091 , -0.03608770834733989
Random destination location:    51.46682374984601 , -0.1556613903628863
Nearest train station to Random Origin: Elverson Road 51.4693 -0.0174
Nearest train station to Random Destination: Clapham Common 51.4618 -

For bikes there were: 0 exact matches.
For bikes there were: 2779 relative matches.
The average time of this BIKE trip is: 16.90788053256567 minutes.
[!] Iteration 12
Random origin location:         51.48425192923328 , -0.15906961619959104
Random destination location:    51.55122727587797 , -0.20447769012961645
Nearest train station to Random Origin: Sloane Square 51.4924 -0.1565
Nearest train station to Random Destination: Kilburn 51.5471 -0.2047
Nearest bike station to Random Origin: Ormonde Gate, Chelsea (51.4869241, -0.1614992)
Nearest bike station to Random Destination: Wellington Road, St. John's Wood (51.5343659, -0.17422870000000001)
The quickest time for this TRAIN trip is  17 minutes.
For bikes there were: 0 exact matches.
For bikes there were: 4709 relative matches.
The average time of this BIKE trip is: 14.824803567636442 minutes.
[!] Iteration 13
Random origin location:         51.582030821173355 , -0.042030223381104695
Random destination location:    51.59912637444388 , -

For bikes there were: 0 exact matches.
For bikes there were: 575 relative matches.
The average time of this BIKE trip is: 17.74608695652174 minutes.
[!] Iteration 24
Random origin location:         51.6101760564276 , -0.1413364112517226
Random destination location:    51.44803771097617 , -0.12565141864441504
Nearest train station to Random Origin: Arnos Grove 51.6164 -0.1331
Nearest train station to Random Destination: Clapham South 51.4527 -0.14800000000000002
Nearest bike station to Random Origin: Allington street, Westminster (51.5806125, -0.2282815)
Nearest bike station to Random Destination: Saltoun Road, Brixton (51.460580900000004, -0.1132589)
The quickest time for this TRAIN trip is  39 minutes.
For bikes there were: 0 exact matches.
For bikes there were: 45 relative matches.
The average time of this BIKE trip is: 14.444444444444445 minutes.
[!] Iteration 25
Random origin location:         51.53181692130525 , -0.14266466515981774
Random destination location:    51.5249921210369

For bikes there were: 0 exact matches.
For bikes there were: 503 relative matches.
The average time of this BIKE trip is: 19.204771371769382 minutes.
[!] Iteration 36
Random origin location:         51.58957178303607 , -0.1915562685854871
Random destination location:    51.41143582326947 , -0.12778436007846808
Nearest train station to Random Origin: Finchley Central 51.6012 -0.1932
Nearest train station to Random Destination: Tooting Broadway 51.4275 -0.168
Nearest bike station to Random Origin: Allington street, Westminster (51.5806125, -0.2282815)
Nearest bike station to Random Destination: Spencer Park, Wandsworth Common (51.45316469999999, -0.169008)
The quickest time for this TRAIN trip is  42 minutes.
For bikes there were: 0 exact matches.
For bikes there were: 48 relative matches.
The average time of this BIKE trip is: 13.5625 minutes.
[!] Iteration 37
Random origin location:         51.43400225398858 , -0.08297463456496783
Random destination location:    51.52279349118809 , -0.

For bikes there were: 0 exact matches.
For bikes there were: 6 relative matches.
The average time of this BIKE trip is: 61.833333333333336 minutes.
[!] Iteration 48
Random origin location:         51.60835179292214 , -0.06175623331985443
Random destination location:    51.43529253237064 , -0.18711568866883238
Nearest train station to Random Origin: Tottenham Hale 51.5882 -0.0594
Nearest train station to Random Destination: Wimbledon Park 51.4343 -0.1992
Nearest bike station to Random Origin: Victoria Park Road, Hackney Central (51.547124200000006, -0.05682049999999999)
Nearest bike station to Random Destination: Neville Gill Close, Wandsworth (51.4543639, -0.1948588)
The quickest time for this TRAIN trip is  44 minutes.
For bikes there were: 0 exact matches.
For bikes there were: 5 relative matches.
The average time of this BIKE trip is: 42.6 minutes.
[!] Iteration 49
Random origin location:         51.573123743041094 , -0.199350665413979
Random destination location:    51.473415682135

In [531]:
dataFrame = pd.DataFrame.from_dict(output, orient="index")
dataFrame

,oRandomLat,oRandomLong,dRandomLat,dRandomLong,oTrain,dTrain,oBike,oBikeCoords,dBike,dBikeCoords,bikeMins,trainMins
0,51.586326,-0.182258,51.421035,-0.146904,East Finchley,Tooting Broadway,"Allington street, Westminster","(51.5806125, -0.2282815)","Spencer Park, Wandsworth Common","(51.45316469999999, -0.169008)",38,13.562500
1,51.461009,-0.036088,51.466824,-0.155661,Elverson Road,Clapham Common,"Napier Avenue, Millwall","(51.4884314, -0.020638099999999996)","Holden Street, Battersea","(51.468861600000004, -0.1575537)",39,60.250000
2,51.526479,-0.065235,51.441664,-0.137227,Shoreditch,Balham,"Bethnal Green Garden, Bethnal Green","(51.526243, -0.0665823)","Clapham Common North Side, Clapham Common","(51.461240999999994, -0.1525693)",25,30.155673
3,51.438537,-0.138344,51.503314,-0.053982,Balham,Wapping,"Spencer Park, Wandsworth Common","(51.45316469999999, -0.169008)","Garnet Street, Shadwell","(51.5077778, -0.05501209999999999)",22,62.000000
4,51.447037,-0.082506,51.491836,-0.026117,Brixton,Mudchute,"Saltoun Road, Brixton","(51.460580900000004, -0.1132589)","Napier Avenue, Millwall","(51.4884314, -0.020638099999999996)",28,40.000000
5,51.597539,-0.058099,51.449153,-0.130765,Tottenham Hale,Clapham South,"Victoria Park Road, Hackney Central","(51.547124200000006, -0.05682049999999999)","Ferndale Road, Brixton.","(51.4627695, -0.12202550000000001)",34,23.677130
6,51.525788,-0.076777,51.468956,-0.037632,Shoreditch,New Cross Gate,"Shoreditch High Street, Shoreditch","(51.5242865, -0.0772422)","Napier Avenue, Millwall","(51.4884314, -0.020638099999999996)",13,21.163288
7,51.605367,-0.172123,51.529678,-0.103940,West Finchley,Angel,"Allington street, Westminster","(51.5806125, -0.2282815)","Chadwell Street, Angel","(51.5304514, -0.10648710000000002)",24,14.692308
8,51.499749,-0.032946,51.519165,-0.184535,Heron Quays,Royal Oak,"Alpha Grove, Millwall","(51.49857739999999, -0.0229211)","Orsett Terrace, Bayswater","(51.51774210000001, -0.1842862)",29,51.833333
9,51.482625,-0.145961,51.419564,-0.113104,Pimlico,Tooting Bec,"Sopwith Way, Battersea Park","(51.482434600000005, -0.14789270000000002)","Saltoun Road, Brixton","(51.460580900000004, -0.1132589)",12,15.146715


In [532]:
dataFrame.to_csv("./computedData.csv")
